In [287]:
# importing libraries
import pandas as pd
from datetime import datetime
import numpy as np
import math

In [171]:
#df = pd.read_csv(r'Data\IMDB.csv')
df_test = df.sample(n=5000000, random_state=1)

In [217]:
df_test = df_test[df_test.startYear != '\\N']
df_test["titleType"] = df_test["titleType"].apply(lambda x: str(x).lower())
df_test["genres"] = df_test["genres"].apply(lambda x: str(x).lower())

In [370]:
# creating a dict file containing movie id as key and date and rating as value in a tuple

test_dict = {"tt0309201": ("20/05/2023", "6"), "tt0016032": ("21/05/2022", "7"),
            "tt1022045": ("22/03/2023", "4"), "tt0531025": ("23/04/2023", "7"),
            "tt0032010": ("24/05/2023", "9"), "tt0034583": ("25/05/2023", "8"),
            "tt0036775": ("26/05/2023", "7"), "tt0836868": ("27/05/2023", "6"),
            "tt0038355": ("28/05/2023", "5"), "tt0038650": ("29/05/2023", "4"),
            "tt0038787": ("30/05/2023", "9"), "tt0039628": ("31/05/2023", "2"),
            "tt9040067": ("01/06/2019", "6"), "tt0640522": ("02/06/2023", "6"),
            "tt7040746": ("03/06/2023", "3"), "tt0040897": ("04/06/2023", "4"),
            "tt8041959": ("05/06/2023", "5"), "tt8041959": ("06/06/2023", "6"),
            "tt8042041": ("07/06/2023", "7"), "tt8042332": ("08/06/2023", "8"),
            "tt8943014": ("09/06/2023", "9"), "tt8044079": ("10/06/2023", "10"),
            "tt8045152": ("11/05/2023", "9"), "tt0046250": ("12/05/2023", "8"),
            "tt0047296": ("15/02/2010", "5"), "tt0047396": ("16/02/2023", "4"),
            "tt0047478": ("17/03/2023", "8"), "tt0347528": ("18/03/2023", "5"),
            "tt0047673": ("19/04/2023", "1"), "tt0947834": ("20/04/2023", "2"),
            "tt1048021": ("21/04/2023", "3"), "tt0948356": ("22/04/2023", "4"),
            }

In [372]:
# dropping all movies that were rated 3 and below
delete_list = []
for key, value in test_dict.items():
    if int(value[1]) <= 3:
        delete_list.append(key)

for key in delete_list:
    del test_dict[key]

In [342]:
# creating a new df to store the required columns
df_input = df.loc[df["tconst"].isin(list(test_dict.keys()))].copy()
df_input.reset_index(inplace=True, drop=True)
df_input["relevance"] = 0

In [343]:
#### Delete this later ####
df_input = df_input[df_input.startYear != '\\N']

In [362]:
# this function decides on the relevance of each movie based on the date and rating

def relevance(test_dict):
    for key, value in test_dict.items():
        date = datetime.strptime(value[0], "%d/%m/%Y")
        rating = value[1]
        relevance = float(rating) /(math.sqrt((abs(int((date - date.today()).days))))+ 0.01)
        df_input["relevance"].loc[df_input["tconst"] == key] = relevance

In [363]:
relevance(test_dict)

C:\Users\Finn\AppData\Local\Temp\ipykernel_3384\3968286732.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input["relevance"].loc[df_input["tconst"] == key] = relevance


In [133]:
# converting genre to lowercase and splitting the string into a list
# in order to process rows with multiple genres
df_input["genres"] = df_input["genres"].apply(lambda x: str(x).lower())
df_input["genres"] = df_input["genres"].apply(lambda x: x.split(","))

In [134]:

# optaining unique genres and creating a dictionary to count the number of movies in each genre
# adding the relevance of each movie to the corresponding genre
genre_list = [item for sublist in df_input["genres"].tolist() for item in sublist]
genre_dict = {genre: 0 for genre in set(genre_list)}

for i in range(len(df_input)):
    temp_list = [df_input["genres"].iloc[i]][0]
    for genre in temp_list:
            genre_dict[genre] += df_input["relevance"].iloc[i]

In [136]:
# converting genre to lowercase and splitting the string into a list
# in order to process rows with multiple genres
df_input["titleType"] = df_input["titleType"].apply(lambda x: str(x).lower())

In [138]:
# optaining unique genres and creating a dictionary to count the number of movies in each genre
# adding the relevance of each movie to the corresponding genre
type_list = df_input["titleType"].tolist()
type_dict = {ttype: 0 for ttype in set(type_list)}

for i in range(len(df_input)):
    temp = [df_input["titleType"].iloc[i]][0]
    type_dict[temp] += df_input["relevance"].iloc[i]

In [140]:
# creating a dictionary for each decade and initializing the value to 0
decade_dict = {"1880-1889" : 0, "1890-1899" : 0, "1900-1909" : 0,"1910-1919" : 0, "1920-1929" : 0,
               "1930-1939" : 0, "1940-1949" : 0, "1950-1959" : 0,"1960-1969" : 0, "1970-1979" : 0,
               "1980-1989" : 0, "1990-1999" : 0, "2000-2009" : 0,"2010-2019" : 0, "2020-2029" : 0
               }

In [180]:
# adding the relevance of each movie to the corresponding decade
for i in range(len(df_input)):
    year = [df_input["startYear"].iloc[i]][0]
    for decade in decade_dict:
        start_year, end_year = map(int, decade.split('-'))
        if start_year <= int(year) <= end_year:
            decade_dict[decade] += df_input["relevance"].iloc[i]

In [144]:
# creating a rating interval and removeing rating with too little votes 
for i in range(len(df_input)):
    if df_input["numVotes"].iloc[i] <= 100:
        df_input["averageRating"].iloc[i] = np.nan

C:\Users\Finn\AppData\Local\Temp\ipykernel_3384\3830930124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input["averageRating"].iloc[i] = np.nan


In [145]:
# creating a dictionary for each rating and initializing the value to 0
df_rating = df_input.dropna(subset=["averageRating"])
rating_list = df_rating["averageRating"].tolist()
rating_dict = {rating: 0 for rating in set(rating_list)}

# adding the relevance of each movie to the corresponding rating
for i in range(len(rating_dict)):
    rating = [df_rating["averageRating"].iloc[i]][0]
    rating_dict[rating] += df_input["relevance"].iloc[i]

# creating a weighted mean for each rating
weighted_mean = sum([value*rating for rating, value in rating_dict.items()]) / sum(rating_dict.values())

In [184]:
# filtering the df based on the rating range and minimum number of votes

df_test = df_test[df_test["numVotes"] > 100]
df_test = df_test[df_test["averageRating"].between(round(weighted_mean,1)-0.5,
                                                   round(weighted_mean,1)+0.5, inclusive=True)]

C:\Users\Finn\AppData\Local\Temp\ipykernel_3384\3594161010.py:4: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df_test = df_test[df_test["averageRating"].between(round(weighted_mean,1)-0.5,


In [149]:
# creating a dictionary for each genre and initializing the value to 0
# assigning a value based on the relevance of each genre
genre_values_dict = {}
y = 3
for i in range(3):
    genre_values_dict[max(genre_dict, key=genre_dict.get)] = y
    genre_dict.pop(max(genre_dict, key=genre_dict.get))
    y -= 1

In [150]:
# creating a dictionary for each type and initializing the value to 0
# assigning a value based on the relevance of each type

type_values_dict = {}
y = 3
for i in range(3):
    type_values_dict[max(type_dict, key=type_dict.get)] = y
    type_dict.pop(max(type_dict, key=type_dict.get))
    y -= 1

In [151]:
# creating a dictionary for each decade and initializing the value to 0
# assigning a value based on the relevance of each type
decade_value_dict = {}
decade_value_dict[max(decade_dict, key=decade_dict.get)] = 3
temp_list = list(decade_dict)
decade_value_dict[temp_list[temp_list.index(max(decade_dict, key=decade_dict.get))-1]] = 1
decade_value_dict[temp_list[temp_list.index(max(decade_dict, key=decade_dict.get))+1]] = 1

In [186]:
df_test.reset_index(inplace=True, drop=True)

In [196]:
# assigning a score to each movie based on the genre, type and decade

df_test["Genre_Score"] = 0
df_test["Type_Score"] = 0
df_test["Decade_Score"] = 0

# assigning a score for the genre(s)
for i in range(len(df_test)):
    temp = df_test.iloc[i]["genres"].split(",")
    for genre in temp:
        genre = str(genre).lower()
        if genre in genre_values_dict.keys():
            df_test.at[i, "Genre_Score"] += genre_values_dict[genre]

# assigning as score for the type
    temp = df_test.iloc[i]["titleType"]
    temp = str(temp).lower()
    if temp in type_values_dict.keys():
        df_test.at[i,"Type_Score"] += type_values_dict[temp]
    
# assigning a score for the decade/year
    year = df_test.iloc[i]["startYear"]
    for decade in decade_value_dict:
        start_year, end_year = map(int, decade.split('-'))
        if start_year <= int(year) <= end_year:
            df_test.at[i,"Decade_Score"] += decade_value_dict[decade]

# calculating the total score
df_test["Total_Score"] = df_test["Genre_Score"] + df_test["Type_Score"] + df_test["Decade_Score"]

In [279]:
# selecting the top 5 movies based on the total score
df_best_score = df_test[df_test.Total_Score == df_test["Total_Score"].max()].sample(5)
try:
    print(f'The follwing movies were found based on your personal preferences: {df_best_score.sample(5)["primaryTitle"].tolist()}')
except ValueError:
    print(f'The follwing movies were found based on your personal preferences: {df_best_score["primaryTitle"].tolist()}')

The follwing movies were found based on your personal preferences: ['The Solitary Cyclist', 'The Cemetery Vote', 'No Way Down', "Brother's Keeper", 'Have His Carcase: Episode One']


In [280]:
# selecting the top 5 movies based on the favorite type and highest score
df_fav_type = df_test[df_test.titleType == list(type_values_dict.keys())[0]]
df_fav_type = df_fav_type[df_fav_type.Total_Score == df_fav_type["Total_Score"].max()]
try:
    print(f'The follwing movies were found based on your favorite Type: {df_fav_type.sample(5)["primaryTitle"].tolist()}')
except ValueError:
    print(f'The follwing movies were found based on your favorite Type: {df_fav_type["primaryTitle"].tolist()}')

The follwing movies were found based on your favorite Type: ['Little Miss Dangerous', 'Into the Shadows', 'Heroes of the Revolution', 'Siege', 'Silver Blaze']


In [281]:
# selecting the top 5 movies based on the favorite genre and highest score
df_fav_genre = df_test[df_test.genres == list(genre_values_dict.keys())[0]]
df_fav_genre = df_fav_genre[df_fav_genre.Total_Score == df_fav_genre["Total_Score"].max()]

try:
    print(f'The follwing movies were found based on your favorite genre: {df_fav_genre.sample(5)["primaryTitle"].tolist()}')
except ValueError:
    print(f'The follwing movies were found based on your favorite genre: {df_fav_genre["primaryTitle"].tolist()}')
    

The follwing movies were found based on your favorite genre: ['Road', 'Dekalog, cztery', 'Grown-Ups', 'Coast to Coast', 'Dekalog, siedem']
